In [2]:
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from sklearn.cluster import SpectralClustering
from torchvision import models, transforms
from PIL import Image
import os
import pandas as pd
import torch

In [4]:
# Function to extract image features using Multi-feature Fusion Algorithm
def extract_features(frame):
    # Ensure that the frame is a valid image
    if frame is None or len(frame.shape) != 3:
        raise ValueError(f"Invalid input frame: {frame}")

    # Convert the frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Calculate 2D histogram in HSV color space
    hsv_hist = cv2.calcHist([hsv_frame], [0, 1], None, [256, 256], [0, 256, 0, 256])

    # Normalize the histogram
    hsv_hist = cv2.normalize(hsv_hist, hsv_hist).flatten()

    return hsv_hist

# Function to calculate cosine similarity between frames
def calculate_similarity(frame1, frame2):
    features1 = extract_features(frame1)
    features2 = extract_features(frame2)
    similarity = cosine_similarity([features1], [features2])[0][0]
    return similarity

# Function to apply Graph Modularity Clustering Algorithm
def modularity_clustering(graph):
    spectral = SpectralClustering(n_clusters=2, affinity='nearest_neighbors', random_state=42)
    labels = spectral.fit_predict(nx.to_numpy_array(graph))
    return labels

# Function to apply skin image processing using ResNet-50
def skin_image_processing(image_path):
    # Implement your skin image processing using ResNet-50 here
    # Placeholder: Using a simple transformation for demonstration
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)
    
    # Load pre-trained ResNet-50 model
    model = models.resnet50(pretrained=True)
    model.eval()
    
    # Placeholder: Forward pass through ResNet-50
    with torch.no_grad():
        output = model(image)

    # Placeholder: Extracting predicted class, replace with your logic
    predicted_class = torch.argmax(output).item()

    return predicted_class

# Function to load metadata from CSV
def load_metadata(metadata_path):
    return pd.read_csv(metadata_path)

# Function to display image with metadata
def display_image_with_metadata(image_path, metadata):
    # Print image path and metadata information
    print(f"Image Path: {image_path}")
    print("Metadata:")
    print(metadata)
    
    # You can also add code here to display the image using a library like OpenCV or PIL
    # For example, to display an image using OpenCV:
    image = cv2.imread(image_path)
    cv2.imshow("Image with Metadata", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Function to process video and integrate with HAM10000 dataset
def process_video(video_path, dataset_path, metadata_path, output_directory):
    # Initialize variables
    keyframes = []
    graph = nx.Graph()
    clusters = []

    # Load metadata from HAM10000 dataset
    metadata = load_metadata(metadata_path)

    # Open video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is successfully opened
    if not cap.isOpened():
        print("Error: Unable to open the video file.")
        return

    # Create directories to store processed frames and keyframes
    processed_frames_dir = os.path.join(output_directory, "processed_frames")
    keyframes_dir = os.path.join(output_directory, "keyframes")

    os.makedirs(processed_frames_dir, exist_ok=True)
    os.makedirs(keyframes_dir, exist_ok=True)

    # Loop through each frame in the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        try:
            # Extract features and add frame to the graph
            features = extract_features(frame)
            graph.add_node(len(graph), frame=frame, features=features)

            # Save processed frame
            frame_filename = f"{len(graph)}.jpg"
            frame_path = os.path.join(processed_frames_dir, frame_filename)
            cv2.imwrite(frame_path, frame)
        except ValueError as e:
            print(f"Error processing frame: {e}")

    # Calculate similarity between frames and add edges to the graph
    for i in range(len(graph)):
        for j in range(i + 1, len(graph)):
            similarity = calculate_similarity(graph.nodes[i]['frame'], graph.nodes[j]['frame'])
            graph.add_edge(i, j, weight=similarity)

    # Apply Graph Modularity Clustering Algorithm
    labels = modularity_clustering(graph)

    # Identify clusters based on modularity scores
    for cluster_id in np.unique(labels):
        cluster_nodes = np.where(labels == cluster_id)[0]
        cluster = graph.subgraph(cluster_nodes)
        clusters.append(cluster)

    # Select keyframes from each cluster based on quality metrics
    for cluster in clusters:
        avg_quality_scores = []
        for node in cluster.nodes():
            # Ensure 'features' key is present in all nodes
            if 'features' not in graph.nodes[node]:
                graph.nodes[node]['features'] = np.zeros(256 * 256)  # Placeholder: Use appropriate size

            # Placeholder: Implement your own quality metric calculation
            quality_score = np.mean(graph.nodes[node]['features'])
            avg_quality_scores.append(quality_score)

        # Select the frame with the highest quality based on quality metrics
        keyframe_index = np.argmax(avg_quality_scores)
        keyframes.append(keyframe_index)

        # Save keyframe
        keyframe_filename = f"{keyframe_index}.jpg"
        keyframe_path = os.path.join(keyframes_dir, keyframe_filename)
        cv2.imwrite(keyframe_path, graph.nodes[keyframe_index]['frame'])

    # Close video file
    cap.release()

    # Process selected keyframes
    for keyframe_index in keyframes:
        keyframe_path = os.path.join(output_directory, "keyframes", f"{keyframe_index}.jpg")

        # Apply skin image processing using ResNet-50
        skin_condition = skin_image_processing(keyframe_path)

        # Display the result
        display_image_with_metadata(keyframe_path, metadata.iloc[keyframe_index])
        print(f"Skin Condition: {skin_condition}")

In [ ]:
# Set the paths and call the main function
video_path = r"C:\Users\Sumit\Desktop\Smart India Hackathon 2023\Backend Deployment Files Repo\Test_skin_video_sample_01.mp4"
dataset_path = r"C:\Users\Sumit\Desktop\Smart India Hackathon 2023\Backend Deployment Files Repo\ISIC-images"
metadata_path = r"C:\Users\Sumit\Desktop\Smart India Hackathon 2023\Backend Deployment Files Repo\metadata.csv"
output_directory = r"C:\Users\Sumit\Desktop\Smart India Hackathon 2023\Backend Deployment Files Repo\ISIC-images\output"

process_video(video_path, dataset_path, metadata_path, output_directory)

C:\Users\Sumit\anaconda3\Lib\site-packages\sklearn\cluster\_spectral.py:688: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(
C:\Users\Sumit\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\Sumit\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Sumit\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behav

Image Path: C:\Users\Sumit\Desktop\Smart India Hackathon 2023\Backend Deployment Files Repo\ISIC-images\output\keyframes\162.jpg
Metadata:
isic_id                                                        ISIC_0024468
attribution               ViDIR Group, Department of Dermatology, Medica...
copyright_license                                                  CC-BY-NC
age_approx                                                             75.0
anatom_site_general                                               head/neck
benign_malignant                                                        NaN
diagnosis                                                 actinic keratosis
diagnosis_confirm_type                                       histopathology
image_type                                                      dermoscopic
lesion_id                                                        IL_6302381
melanocytic                                                           False
sex                      